In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os
import sys
from sqlalchemy import create_engine
from importlib import reload


In [2]:
import os, sys
from importlib import reload

MODULES_DIR = os.path.join(os.getcwd(), "..", "..", "modules")
sys.path.append(os.path.abspath(MODULES_DIR))

# 1. importe os módulos (não as funções)
import prefilter
import embedder
import help
import prompt_builder

# 2. recarregue apenas os módulos que você editou
reload(prefilter)
reload(embedder)
reload(help)
reload(prompt_builder)

# 3. agora sim importe as funções atualizadas
from prefilter import prefilter_results
from embedder import HuggingFaceEmbedder
from help import estimate_tokens
from prompt_builder import build_prompt


C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()

True

# Pre-filter ddgo

### Get retrieval data

In [4]:
USER = os.getenv("POSTGRES_USER")
PASSWORD = os.getenv("POSTGRES_PASSWORD")
DB = os.getenv("POSTGRES_DB")
PORT = os.getenv("POSTGRES_PORT")
HOST = os.getenv("POSTGRES_HOST", "localhost")

# Criar engine SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}")


In [5]:
df = pd.read_sql("SELECT * FROM retrieved_news_ddgo", engine)
df

,id,search_title,original_title,refined_title,snippet,link,domain,shuffle_id
0,101,Supreme Court Justice Ginsburg 'regrets' Trump...,Supreme Court Justice Ginsburg ' regrets ' Tru...,Supreme Court Justice Ginsburg ' regrets ' Tru...,Supreme Court justices generally shy away from...,https://www.yahoo.com/news/supreme-court-justi...,yahoo.com,0
1,102,Supreme Court Justice Ginsburg 'regrets' Trump...,Ginsburg regrets ‘ill-advised’ criticism of Tr...,Ginsburg regrets ‘ill-advised’ criticism of Tr...,"Jul 14, 2016 · WASHINGTON — Supreme Court Just...",https://www.pbs.org/newshour/politics/ginsburg...,pbs.org,0
2,103,Supreme Court Justice Ginsburg 'regrets' Trump...,Ginsburg apologizes for “ill-advised” criticis...,Ginsburg apologizes for “ill-advised” criticis...,"Jul 14, 2016 · Conceding her outspoken critici...",https://apnews.com/united-states-government-su...,apnews.com,0
3,104,Supreme Court Justice Ginsburg 'regrets' Trump...,U.S. Supreme Court Justice Ginsburg 'regrets' ...,U.S. Supreme Court Justice Ginsburg 'regrets' ...,"Jul 14, 2016 · Ginsburg, the 83-year-old senio...",https://www.reuters.com/article/world/us-supre...,reuters.com,0
4,105,Supreme Court Justice Ginsburg 'regrets' Trump...,Ginsburg regrets comments on Trump - Paris Pos...,Ginsburg regrets comments on Trump - Paris Pos...,"Jul 14, 2016 · WASHINGTON (AP) — Supreme Court...",https://www.parispi.net/stories/ginsburg-regre...,parispi.net,0
...,...,...,...,...,...,...,...,...
99241,99342,"Globalization, migration fears reawaken German...",HEIMAT / watan - ONCURATING,HEIMAT / watan - ONCURATING,"But this very common interest in Heimat , I di...",https://www.on-curating.org/issue-30-reader/he...,on-curating.org,4999
99242,99343,"Globalization, migration fears reawaken German...",worldmigrationreport.iom.int/msite/wmr-2024-in...,worldmigrationreport.iom.int/msite/wmr-2024-in...,World Migration Report 2024 (Si apre in una nu...,https://worldmigrationreport.iom.int/msite/wmr...,worldmigrationreport.iom.int,4999
99243,99344,"Globalization, migration fears reawaken German...",peoplemovin - A visualization of migration flows,peoplemovin - A visualization of migration flows,The number of international migrants grew stea...,https://peoplemov.in/,peoplemov.in,4999
99244,99345,"Globalization, migration fears reawaken German...",The Heimat Abroad: The Boundaries of Germannes...,The Heimat Abroad: The Boundaries of Germannes...,"In the journal’s pages, German -speaking colon...",https://epdf.pub/the-heimat-abroad-the-boundar...,epdf.pub,4999


### Filter Data by suffle_id

In [6]:
df_filt = df[df['shuffle_id'].between(0, 1999)]

In [7]:
df_filt['search_title'].nunique()

2000

In [8]:
df_filt.groupby('shuffle_id').size().describe()

count    2000.000000
mean       19.876500
std         1.060099
min         2.000000
25%        20.000000
50%        20.000000
75%        20.000000
max        20.000000
dtype: float64

### NER

In [9]:
from transformers import pipeline

class EntityComparator:
    def __init__(self, model_name="dslim/bert-base-NER", grouped=True):
        """
        Inicializa o pipeline de NER.
        """
        self.ner = pipeline(
            "ner",
            model=model_name,
            grouped_entities=grouped
        )

    def extract_entities(self, text):
        """
        Extrai entidades nomeadas como um conjunto de strings.
        """
        ents = self.ner(text)
        return {e["word"] for e in ents}

    def ner_score(self, text1, text2):
        """
        Retorna score entre 0 e 1 baseado na fração de entidades de text1
        que aparecem em text2.

        Regra:
        - score = interseção / total_entidades_text1
        - se text1 não tiver entidades → score = 1
        """
        ents1 = self.extract_entities(text1)
        ents2 = self.extract_entities(text2)

        # Caso especial
        if len(ents1) == 0:
            return {
                "score": 0.5,
                "ents_text1": [],
                "ents_text2": [],
                "intersection": []
            }
        
        intersection = ents1.intersection(ents2)

        score = len(intersection) / len(ents1)

        return {
            "score": score,
            "ents_text1": list(ents1),
            "ents_text2": list(ents2),
            "intersection": list(intersection)
        }

cmp = EntityComparator()

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\pipelines\token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="Aggr

### Filter retrievel data

In [10]:
embedder = HuggingFaceEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
top_x = 10  # número de top resultados que queremos

# summary_rows = []
# filtered_by_title = {}

# for idx, title in enumerate([' Female GOP Leader RAGE QUITS Over Party’s Support For Trump’s Serial Sexual Assaults']):
#     print(idx)
#     subset = df_filt[df_filt['search_title'] == title]
#     total_before = len(subset)
    
#     subset = subset.drop_duplicates(subset=["original_title"])
#     duplicates_removed = total_before - len(subset)
#     shuffle_id_val = subset["shuffle_id"].iloc[0]

#     results = [
#         {
#             "refined_title": row["refined_title"],
#             "original_title": row["original_title"],
#             "domain": row["domain"],
#             "snippet": row["snippet"],
#             "search_title": row["search_title"],
#             "shuffle_id": row["shuffle_id"],
#         }
#         for _, row in subset.iterrows()
#     ]

#     # aplica filtro de similaridade
#     filtered_results = prefilter_results(
#         results=results,
#         original_title=title,
#         embedder=embedder,
#         credible_domains_file='../../out/credible_sources.txt'
#     )

#     for result in filtered_results:
#         details = cmp.ner_score(title, result['refined_title'])
#         result['query'] = title
#         result['NER_score'] = details['score']
#         result['NER_intersection'] = details['intersection']
#         result['ents_text1'] = details['ents_text1']
#         result['ents_text2'] = details['ents_text2']
#         result['NER_count'] = len(details['ents_text1'])

#     # contar removidos pelo filtro de similaridade
#     non_similar_removed = len(subset) - len(filtered_results)

#     # agora o total final após duplicatas + filtro de similaridade
#     totat_after = len(filtered_results)

#     # salva resultados filtrados
#     filtered_by_title[title] = filtered_results

#     # adiciona linha ao summary
#     summary_rows.append({
#         "search_title": title,
#         "shuffle_id": shuffle_id_val,
#         "total_before": total_before,
#         "duplicates_removed": duplicates_removed,
#         "non_similar_removed": non_similar_removed,
#         "totat_after": totat_after,
#     })

In [12]:
import json

# with open("resultados.json", "w", encoding="utf-8") as f:
#     json.dump(filtered_by_title, f, indent=4, ensure_ascii=False)

with open("resultados.json", "r", encoding="utf-8") as f:
    filtered_by_title = json.load(f)

In [13]:
len(filtered_by_title)

2000

In [14]:
import numpy as np

alpha = 0.8
beta  = 0.2

final = {}

for k,values in filtered_by_title.items():
    for value in values:
        value["score"] = alpha*value["similarity"] + beta*value["NER_score"]

    matched = values[:]
    matched_085 = [item for item in values if item["score"] >= 0.85]
    matched_080 = [item for item in values if item["score"] >= 0.80]
    matched_070 = [item for item in values if item["score"] >= 0.70]
    
    if matched_085: matched = matched_085[:]        
    elif matched_080: matched = matched_080[:]
    elif matched_070: matched = matched_070[:]

    matched = sorted(matched, key=lambda x: x['score'], reverse=True)[:top_x]
    final[k] = matched
    
len(final)

2000

## Build Prompts

In [15]:
def generate_prompts(filtered_by_title, mode='test1'):
    rows = []
    for title, filtered_results in filtered_by_title.items():

        shuffle_id = filtered_results[0].get("shuffle_id") if filtered_results else df_filt.set_index('search_title').to_dict()['shuffle_id'][title]

        prompt = build_prompt(
            mode=mode,
            title_to_check=title,
            results_filtered=filtered_results
        )

        info = {
            "search_title": title,
            "shuffle_id": shuffle_id,
            "prompt": prompt,
            "num_results": len(filtered_results),
            "approx_tokens": estimate_tokens(prompt),
            "prompt_length_chars": len(prompt),
        }
        rows.append(info)

    df_prompts = pd.DataFrame(rows)
    return df_prompts

In [ ]:
df_test1 = generate_prompts(final, mode="test1")
df_test1 = df_test1[~df_test1['shuffle_id'].isna()]
print(len(df_test1.search_title.unique()))
df_test1.to_sql(
    "teste_ner_ddgo_prompts",
    engine,
    if_exists="replace",
    index=False
)

2000


1000